## Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import datetime
import itertools
from collections import defaultdict

import math
import numpy as np
import pandas as pd
from scipy.stats import shapiro
import scipy.stats as stats

import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNetCV, ElasticNet
from xgboost import XGBRegressor
import catboost as cb
import lightgbm as lgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, KFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler,OrdinalEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error, make_scorer
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.compose import make_column_selector as selector

## Configs

In [ ]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

## Load Data

In [ ]:
dir_path = '/kaggle/input/store-sales-time-series-forecasting/'

oil_df = pd.read_csv(dir_path + 'oil.csv')
holidays_df = pd.read_csv(dir_path + 'holidays_events.csv')
stores_df = pd.read_csv(dir_path + 'stores.csv')
train_df = pd.read_csv(dir_path + 'train.csv')
test_df = pd.read_csv(dir_path + 'test.csv')
trnsctns_df = pd.read_csv(dir_path + 'transactions.csv')
submissions_df = pd.read_csv(dir_path + 'sample_submission.csv')

In [ ]:
oil_df.head()

In [ ]:
holidays_df.head()

In [ ]:
stores_df.head()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
trnsctns_df.head()

In [ ]:
submissions_df.head()

## Data Pre-processing

#### Change 'date' column type

In [ ]:
oil_df['date'] = pd.to_datetime(oil_df['date'], format = "%Y-%m-%d")
holidays_df['date'] = pd.to_datetime(holidays_df['date'], format = "%Y-%m-%d")
trnsctns_df['date'] = pd.to_datetime(trnsctns_df['date'], format = "%Y-%m-%d")
train_df['date'] = pd.to_datetime(train_df['date'], format = "%Y-%m-%d")
test_df['date'] = pd.to_datetime(test_df['date'], format = "%Y-%m-%d")

#### Fill missing values in datasets

In [ ]:
train_data_strt_dt = train_df['date'].min()
train_data_end_dt = train_df['date'].max()
train_dt_rnge = pd.date_range(start=train_data_strt_dt, end=train_data_end_dt)
train_missing_dts = train_dt_rnge.difference(train_df['date'])

test_data_strt_dt = test_df['date'].min()
test_data_end_dt = test_df['date'].max()
test_dt_rnge = pd.date_range(start=test_data_strt_dt, end=test_data_end_dt)
test_missing_dts = test_dt_rnge.difference(test_df['date'])

print(f"Missing dates in training set: {train_missing_dts}")
print(f"Missing dates in test set: {test_missing_dts}")

Create a Multi-index variable

In [ ]:
multi_index = pd.MultiIndex.from_product([pd.date_range(train_data_strt_dt, train_data_end_dt),
                                          train_df.store_nbr.unique(),
                                          train_df.family.unique()],
                                         names=['date','store_nbr','family'],)
train_df = train_df.set_index(['date','store_nbr','family']).reindex(multi_index).reset_index()

train_df.head()

Fill missing values with 0s

In [ ]:
train_df[['sales','onpromotion']] = train_df[['sales','onpromotion']].fillna(0.)

# Apply linear interpolation to the "id" column to estimate the missing values based on the linear relationship between adjacent data points. 
train_df.id = train_df.id.interpolate(method="linear")

Add an additional column in both training and test sets to separate those two

In [ ]:
train_df['test'] = 0
test_df['test'] = 1

#### Oil Data - Re-index by adding missing dates

In [ ]:
# Create a date range from the start of training data to the end of test data
date_range = pd.date_range(train_data_strt_dt, test_data_end_dt)

# Create a DataFrame with the date range
date_df = pd.DataFrame({'date': date_range})

# Merge the date_df with oil_df using an outer join
oil_df = oil_df.merge(date_df, on = 'date', how = 'outer')

# Sort the DataFrame by date and reset the index
oil_df = oil_df.sort_values('date', ignore_index=True)

# fill missing values using linear interpolation
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].interpolate(method="linear", limit_direction="both")

oil_df.head()

#### Transactions Data - Fill in the missing values for transactions using interpolation, except for days with zero sales

In [ ]:
# Calculate the number of unique store numbers
num_store = train_df['store_nbr'].nunique()

# Calculate the number of days in the training period
train_len = (train_data_end_dt - train_data_strt_dt).days + 1

# Calculate the number of records where sales are zero
num_zero_sales = (train_df.groupby(["date", "store_nbr"])['sales'].sum() == 0).sum()

# Calculate the total number of expected records
total_rec = num_store * train_len

# Calculate the current number of records
curr_rec = len(trnsctns_df.index)

# Calculate the number of missing records
missing_rec = total_rec - curr_rec - num_zero_sales

# Total sales for each store
store_sales = train_df.groupby(["date", "store_nbr"]).sales.sum().reset_index()

# Re-index transaction data
trnsctns_df = trnsctns_df.merge(store_sales, on=["date", "store_nbr"],how="outer").sort_values(["date", "store_nbr"],ignore_index=True)


# Fill missing values with 0s for days with zero sales
trnsctns_df.loc[trnsctns_df.sales.eq(0), "transactions"] = 0

# Drop the "sales" column
trnsctns_df = trnsctns_df.drop(columns=["sales"])

# Fill remaining missing values using linear interpolation within each "store_nbr" group
trnsctns_df["transactions"] = trnsctns_df.groupby("store_nbr")["transactions"].transform(lambda x: x.interpolate(method="linear", limit_direction="both"))

#### Holidays Data - Remove transferred holidays, and separate work days from the main DataFrame for further analysis

In [ ]:
# Define a function to process holiday descriptions
def process_holiday(s):
    # Check if "futbol" is in the description; if so, return "futbol"
    if "futbol" in s:
        return "futbol"
    
    # Create a list of words to remove based on cities and states
    to_remove = list(set(stores_df['city'].str.lower()) | set(stores_df['state'].str.lower()))
    
    # Iterate through the list of words and remove them from the description
    for w in to_remove:
        s = s.replace(w, "")
    
    # Return the processed description
    return s

In [ ]:
# Clean and process the 'description' column in the holidays_data DataFrame
holidays_df['description'] = holidays_df.apply(lambda x: x['description'].lower().replace(x['locale_name'].lower(), ""), axis=1).apply(process_holiday).replace(r"[+-]\d+|\b(de|del|traslado|recupero|puente|-)\b", "", regex=True).replace(r"\s+|-", " ", regex=True).str.strip()

# Remove transferred holidays from the DataFrame
holidays_df = holidays_df[holidays_df['transferred'].eq(False)]

# Extract and process work days
work_days = holidays_df[holidays_df['type'].eq("Work Day")]
work_days = work_days[["date", "type"]].rename(columns={"type": "work_day"}).reset_index(drop=True)

# Convert the 'work_day' column to binary values (1 for work days, 0 for others)
work_days['work_day'] = work_days['work_day'].notna().astype(int)

# Remove work days from the main holidays_data DataFrame
holidays_df = holidays_df[holidays_df['type']!="Work Day"].reset_index(drop=True)

holidays_df.head()

Local holidays at city level with dummy variables for descriptions

In [ ]:
# Filter local holidays at the city level
local_holidays = holidays_df[holidays_df['locale'].eq("Local")]

# Select relevant columns, rename 'locale_name' to 'city', and reset the index
local_holidays = local_holidays[["date", "locale_name", "description"]].rename(columns={"locale_name": "city"}).reset_index(drop=True)

# Remove duplicated rows
local_holidays = local_holidays[~local_holidays.duplicated()]

# Create dummy variables for 'description' and prefix them with "loc"
local_holidays = pd.get_dummies(local_holidays, columns=["description"], prefix="loc")

# Display the resulting DataFrame
local_holidays.head()

Regional holidays are filtered and processed to include a binary column indicating whether the description contains "provincializacion."

In [ ]:
# Filter regional holidays
regional_holidays = holidays_df[holidays_df['locale'].eq("Regional")]

# Select relevant columns and rename 'locale_name' to 'state' and 'description' to 'provincializacion'
regional_holidays = regional_holidays[["date", "locale_name", "description"]].rename(columns={"locale_name": "state", "description": "provincializacion"}).reset_index(drop=True)

# Create a binary column 'provincializacion' based on the presence of the word "provincializacion" in descriptions
regional_holidays['provincializacion'] = regional_holidays['provincializacion'].eq("provincializacion").astype(int)

# Return the resulting DataFrame
regional_holidays.head()

National holidays are filtered, processed, and organized for further analysis.

In [ ]:
# Filter national holidays
national_holidays = holidays_df[holidays_df['locale'].eq("National")]

# Select relevant columns and reset the index
national_holidays = national_holidays[["date", "description"]].reset_index(drop=True)

# Remove duplicated rows
national_holidays = national_holidays[~national_holidays.duplicated()]

# Create dummy variables for 'description' and prefix them with "nat"
national_holidays = pd.get_dummies(national_holidays, columns=["description"], prefix="nat")

# Group national holidays that fall on the same date and sum the binary values
national_holidays = national_holidays.groupby("date").sum().reset_index()

# Rename columns for visualization purposes
national_holidays = national_holidays.rename(columns={"nat_primer grito independencia": "nat_primer grito"})

# Return the resulting DataFrame
national_holidays.head()

#### Pivot and reshape data into time series format for Sales, Transactions, and Promotions.

In [ ]:
# Pivot the train_data DataFrame to create a time series of sales data
sales_ts = pd.pivot_table(train_df, values="sales", index="date", columns=["store_nbr", "family"])

# Pivot the transactions_data DataFrame to create a time series of transaction data
tr_ts = pd.pivot_table(trnsctns_df, values="transactions", index="date", columns="store_nbr")

# Pivot the train_data DataFrame to create a time series of promotion data
promo_ts = pd.pivot_table(train_df, values="onpromotion", index="date", columns=["store_nbr", "family"])

In [ ]:
sales_ts.head()

#### Create master dataset for time series analysis

In [ ]:
# scale target series
scaler = MinMaxScaler()
sales_ts_scaled = sales_ts.copy()
sales_ts_scaled[sales_ts_scaled.columns] = scaler.fit_transform(sales_ts_scaled)

# convert back to long form and add the holiday columns
holiday_sales_merged = sales_ts_scaled.melt(
    value_name="sales", ignore_index=False,).reset_index().\
    merge(stores_df, on="store_nbr", how="left").\
    merge(work_days, on="date", how="left").\
    merge(local_holidays, on=["date", "city"], how="left").\
    merge(regional_holidays, on=["date", "state"], how="left").\
    merge(national_holidays, on="date", how="left").\
    fillna(0)

# include dummy variable for dates without any holidays
holiday_list = [col for col in holiday_sales_merged if col.startswith(("loc_", "nat_", "provincializacion"))]
holiday_sales_merged["no_holiday"] = holiday_sales_merged[holiday_list].sum(axis=1).eq(0).astype(int)

holiday_sales_merged.head()

Merge data

In [ ]:
# Define a list of selected national holidays with larger impacts on sales
selected_holidays = ["nat_terremoto", "nat_navidad", "nat_dia la madre", "nat_dia trabajo",
                     "nat_primer dia ano", "nat_futbol", "nat_dia difuntos"]

# Select only the columns related to the selected national holidays
keep_national_holidays = national_holidays[["date", *selected_holidays]]


# Concatenate the train and test data along the rows
data = pd.concat([train_df, test_df], axis=0, ignore_index=True)

# Merge the 'stores_data' DataFrame on the 'store_nbr' column
data = data.merge(stores_df, on=["store_nbr"])

# Merge the 'oil_data' DataFrame on the 'date' column, with a left join
data = data.merge(oil_df, on=["date"], how="left")

# Merge the 'transactions_data' DataFrame on the 'date' and 'store_nbr' columns, with a left join
data = data.merge(trnsctns_df, on=["date", 'store_nbr'], how="left")

# Merge the 'work_days' DataFrame on the 'date' column, with a left join
data = data.merge(work_days, on="date", how="left")

# Merge the 'keep_national_holidays' DataFrame on the 'date' column, with a left join
data = data.merge(keep_national_holidays, on=["date"], how="left")

# Sort the resulting dataset by 'date', 'store_nbr', and 'family' columns
data = data.sort_values(["date", "store_nbr", "family"], ignore_index=True)


# The last section of code fills missing values in the 'work_day' and selected holiday columns with 0:
data[["work_day", *selected_holidays]] = data[["work_day", *selected_holidays]].fillna(0)

data.head()

## Feature Engineering

Add new columns that provide temporal information about the data

In [ ]:
# Select the date, days of the week, hours, month (not used in calculations)
data['day_of_week'] = data.date.dt.dayofweek
data['day_of_year'] = data.date.dt.dayofyear
data['day_of_month'] = data.date.dt.day
data['year'] = data.date.dt.year
data['month'] = data.date.dt.month

# Seasons: 0-winter; 1-spring; 2-summer; 3-fall
data["season"] = np.where(data.date.dt.month.isin([12, 1, 2]), 0, 1)
data["season"] = np.where(data.date.dt.month.isin([3, 4, 5]), 1, data["season"])
data["season"] = np.where(data.date.dt.month.isin([6, 7, 8]), 2, data["season"])
data["season"] = np.where(data.date.dt.month.isin([9, 10, 11]), 3, data["season"])

In the EDA, it was found that the sales across stores are distributed unevenly. If there are any memory limitations, we will consider only a part of the stores

In [ ]:
#Let's keep data for 1-20 stores, due to the lack of memory
#data = data.loc[data['store_nbr'].isin(list(range(1, 19)))]

In [ ]:
# Create a copy of the original data
data_analyses = data.copy()

# Specify the target variable
target = 'sales'

# Separate the data into training and testing sets based on the 'test' column
train = data_analyses.loc[data_analyses['test'] == 0]
test = data_analyses.loc[data_analyses['test'] == 1]

train.head()

In [ ]:
# Rolling Summary Stats Features
#A rolling mean is simply the mean of a certain number of previous periods in a time series.
for i in [16,17,18,19,20,21,22,46,76,106,365, 730]:
    data_analyses["sales_roll_mean_"+str(i)]=data_analyses.groupby(["store_nbr", "family"])['sales'].rolling(i).mean().shift(1).values

#### Lag/ Shifted Features

In [ ]:
data_analyses.sort_values(by=['store_nbr', 'family', 'date'], axis=0, inplace=True)

In [ ]:
def lag_features(dataframe, lags, groups = ["store_nbr", "family"], target = "sales", prefix = ''):
    dataframe = dataframe.copy()
    for lag in lags:
        dataframe[prefix + str(lag)] = dataframe.groupby(groups)[target].transform(lambda x: x.shift(lag))
    return dataframe

In [ ]:
#Let's create lags
data_analyses = lag_features(data_analyses, 
                             lags = [16,17,18,19,20,21,22,46,76,106,365, 730],
                             groups = ["store_nbr", "family"], target = 'sales', 
                             prefix = 'sales_lag_')
data_analyses.head(20)

In [ ]:
# Remove the most correlated features
def drop_cor(dataframe, name, index):
    ind = dataframe[dataframe.columns[dataframe.columns.str.contains(name)].tolist()+[
        "sales"]].corr().sales.sort_values(ascending = False).index[1:index]
    ind = dataframe.drop(ind, axis = 1).columns[dataframe.drop(ind, axis = 1).columns.str.contains(name)]
    dataframe.drop(ind, axis = 1, inplace = True)

drop_cor(data_analyses, "sales_lag", 6)

#### Train/ Validation Split

In [ ]:
# Dataframe must be sorted by date because of Time Series Split 
data_analyses = data_analyses.sort_values("date").reset_index(drop=True)

# Let's bring all the columns into a single form to avoid further errors
data_analyses.columns = [column.replace(" ", "_") for column in data_analyses.columns]

# Define the columns that will be further used in the analysis
features = [col for col in data_analyses.columns if col not in ['date', 'id', "sales", 'transactions',
                                                                'day_of_week','day_of_year','day_of_month',
                                                                'year', 'month', 'season','test']]

# Fill NA for all columns with 0 or appropriate empty value based on data type
for col in data_analyses.columns:
    fill_value = 0 if data_analyses[col].dtype in [int, float] else pd.NA
    data_analyses[col] = data_analyses[col].fillna(fill_value)

# Make the data readable
data_analyses['store_nbr'] = data_analyses['store_nbr'].apply(lambda x: (f"store_nbr_{x}"))
data_analyses['cluster'] = data_analyses['cluster'].apply(lambda x: (f"cluster_{x}"))
data_analyses['type'] = data_analyses['type'].apply(lambda x: (f"type_{x}"))
data_analyses['city'] = data_analyses['city'].apply(lambda x: (f"city_{x.lower()}"))
data_analyses['state'] = data_analyses['state'].apply(lambda x: (f"state_{x.lower()}"))

data_analyses.head()

In [ ]:
# Divide the data into training and validation and use them at the stage of identifying important features
train_f = data_analyses.copy()
val_f = data_analyses.copy()

train_f = train_f.loc[(train_f["date"] < "2017-01-01"), :]
val_f = val_f.loc[(val_f["date"] >= "2017-01-01") & (val_f["date"] < "2017-08-16"), :]

Y_train = train_f['sales']
X_train = train_f[features]

Y_val = val_f['sales']
X_val = val_f[features]

# Define object columns
object_cols = X_train.loc[:,X_train.dtypes==object].columns
object_cols = list(object_cols)
cols_for_le = object_cols 
cols_for_le = [list(X_train.columns).index(col) for col in cols_for_le]

# Transform categorical features 
t = [('MeanTargetEncoder', TargetEncoder(), cols_for_le)]
col_transform = ColumnTransformer(transformers=t)
col_transform.set_output(transform="pandas")
X_trans_tr =col_transform.fit_transform(X_train,Y_train)
X_val_tr =col_transform.fit_transform(X_val,Y_val)

Y_train.shape, X_trans_tr.shape, Y_val.shape, X_val_tr.shape

#### Feature Importance

In [ ]:
for c in object_cols:
    X_train[c] = X_train[c].astype('category')
    X_val[c] = X_val[c].astype('category')

In [ ]:
# SMAPE: Symmetric mean absolute percentage error
def smape(preds, target):
    smape_val=1/len(target) * np.sum(2 * np.abs(preds-target) / (np.abs(target) + np.abs(preds))*100)
    return smape_val

In [ ]:
first_model = lgb.LGBMRegressor(random_state=384).fit(X_train, Y_train,
                                                      eval_metric= lambda y_true,
                                                      y_pred: [mean_squared_error(y_true, y_pred)],
                                                      categorical_feature = object_cols)

print("TRAIN SMAPE:", smape(Y_train, first_model.predict(X_train)))
print("VALID SMAPE:", smape(Y_val, first_model.predict(X_val)))

In [ ]:
def plot_lgb_importances(model, plot=False, num=120):
    # SKLEARN API
    gain = model.booster_.feature_importance(importance_type='gain')
    feat_imp = pd.DataFrame({'feature': model.feature_name_,
                             'split': model.booster_.feature_importance(importance_type='split'),
                             'gain': 100 * gain / gain.sum()}).sort_values('gain', ascending=False)
    if plot:
        plt.figure(figsize=(10, 10))
        sns.set(font_scale=1)
        sns.barplot(x="gain", y="feature", data=feat_imp[0:25])
        plt.title('feature')
        plt.tight_layout()
        plt.show()
    else:
        print(feat_imp.head(num))
        return feat_imp

feature_imp_df = plot_lgb_importances(first_model, num=200)

In [ ]:
plot_lgb_importances(first_model, plot=True, num=10)

In [ ]:
# feature importance
cols = feature_imp_df[feature_imp_df.gain > 0.015].feature.tolist()
print("Independent Variables:", len(cols))

In [ ]:
X_train[cols]

## Model Development

In [ ]:
train = data_analyses.loc[data_analyses['test'] == 0]
test = data_analyses.loc[data_analyses['test'] == 1]

X = train[cols]
y = train[target]

In [ ]:
num_folds = 5
tscv = TimeSeriesSplit(n_splits=num_folds)

#creating dictionaries to record results
mse_scores = defaultdict(list)
rmse_scores = defaultdict(list)
r2_scores = defaultdict(list)
mae_scores = defaultdict(list)
mape_scores = defaultdict(list)
smape_scores = defaultdict(list)
models = defaultdict(list)

# Metrics used to evaluate models
def metrics_regression(y_true, y_pred):
    # MSE
    mse = mean_squared_error(y_true, y_pred) #!
    
    # RMSE (Root Mean Square Error)
    rmse = math.sqrt(mse)
    
    # R^2
    r2 = r2_score(y_true, y_pred)
    
    # MAE(mean absolute error)
    mae = mean_absolute_error(y_true, y_pred) #!
    
    # MAPE(mean absolute percentage error)
    mape = mean_absolute_percentage_error(y_true, y_pred) #!
    
    #SMAPE (symmetric mean absolute percentage error)
    smape = 1/len(y_true) * np.sum(2 * np.abs(y_pred-y_true) / (np.abs(y_true) + np.abs(y_pred))*100)
    
    return mse,rmse,r2,mae,mape,smape

### LGBM Regressor

In [ ]:
%%time
for fold_idx, (train_index, test_index) in enumerate(tscv.split(train)):
    print(f"Fold {fold_idx + 1}")

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model_lgbm = lgb.LGBMRegressor(random_state=42,n_estimators=10000, colsample_bytree=0.5,
                                   device_type='gpu', verbose=3)
    
    object_cols = list(X_train.loc[:,X_train.dtypes==object].columns)
    
    for c in object_cols:
        X_train[c] = X_train[c].astype('category')
        X_test[c] = X_test[c].astype('category')

    model_lgbm.fit(X_train, y_train,
                   categorical_feature = object_cols)

    y_pred_LGBM = model_lgbm.predict(X_test)

    models['lgbm'].append(model_lgbm)
    
    mse_LGBM,rmse_LGBM,r2_LGBM,mae_LGBM,mape_LGBM,smape_LGBM = metrics_regression(y_test, y_pred_LGBM)

    mse_scores['lgbm'].append(mse_LGBM)
    rmse_scores['lgbm'].append(rmse_LGBM)
    r2_scores['lgbm'].append(r2_LGBM)
    mae_scores['lgbm'].append(mae_LGBM)
    mape_scores['lgbm'].append(mape_LGBM)
    smape_scores['lgbm'].append(smape_LGBM)
    
    #print(f"\t Score for LGBM: {mse_scores,rmse_scores, r2_scores,mae_scores,mape_scores,smape_scores}")
    print('*'*60)

print(f"\t\t Mean MSE \n\t LGBM: {np.mean(mse_scores['lgbm'])}")
print(f"\t\t Mean RMSE \n\t LGBM: {np.mean(rmse_scores['lgbm'])}")
print(f"\t\t Mean SMAPE \n\t LGBM: {np.mean(smape_scores['lgbm'])}")
print(f"\t\t Mean R2 \n\t LGBM: {np.mean(r2_scores['lgbm'])}")
print(f"\t\t Mean MAE \n\t LGBM: {np.mean(mae_scores['lgbm'])}")
print(f"\t\t Mean MAPE \n\t LGBM: {np.mean(mape_scores['lgbm'])}")
print('*'*60)

## Submission

In [ ]:
result = model_lgbm.predict(test[cols])
sample_submission = pd.DataFrame({'id':test.id,'sales':result}).set_index('id')
sample_submission

In [ ]:
sample_submission.to_csv('submission.csv', index=False)